In [1]:
import numpy as np
from matplotlib import pyplot as plt

# model spec
- genes $G_1,G_2,G_3$
- mRNA $M_i$
- protein $P_i$

$$
G_i \to \xrightarrow{\alpha_0 + \alpha K^n/(K^n+P_j^n)} M_i, \quad M_i \xrightarrow{\beta} M_i + P_i, \quad P_i \xrightarrow{\beta} \empty, \quad M_i\xrightarrow{\gamma} \empty
$$
where $j = i-1 \mod 3$

In [7]:
class Repressilator:
    '''
    see the paper "Multiﬁdelity multilevel Monte Carlo to accelerate approximate Bayesian parameter inference for partially observed stochastic processes"
    for a description of Repressilator. 
    '''
    def __init__(self, initial_population,  leakage_transcription_rate, free_transcription_rate, Hill_coefficient, excluding_leakage, protein_translation_and_degradation_rate, mRNA_degradation_rate):
        
        self.X = initial_population
        self.alpha0 = leakage_transcription_rate
        self.alpha = free_transcription_rate - leakage_transcription_rate
        self.n = Hill_coefficient
        self.K = excluding_leakage
        self.beta = protein_translation_and_degradation_rate
        self.gamma = mRNA_degradation_rate
        
        self.chemicals = 'M1 M2 M3 P1 P2 P3'
        
        self.reactions = [
            ({},            {'M1':1}),
            ({},            {'M2':1}),
            ({},            {'M3':1}),
            ({'M1':1},      {'M1':1,'P1':1}),
            ({'M2':1},      {'M2':1,'P2':1}),
            ({'M3':1},      {'M3':1,'P3':1}),
            ({'P1':0},      {}),
            ({'P2':0},      {}),
            ({'P3':0},      {}),
            ({'M1':0},      {}),
            ({'M2':0},      {}),
            ({'M3':0},      {}),
        ]
    
    def reaction_rates(self, X, theta = None):
        
            
            
        

SyntaxError: invalid syntax (1184300388.py, line 19)